## Estimating Future Water Usage

This section estimates the future water usage based on historical data and predictive modeling.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import pandas as pd

def load_data_from_csv(
    window_size: int,
    csv_path: str = "../assets/plant_watering_data.csv",
    add_watering_flag: bool = True
    ):
    data = pd.read_csv(csv_path, parse_dates=["time", "watered_at"])
    data = data.sort_values("time")

    data["watered"] = data["pump_status"].astype(int)

    feature_cols = ["tank_level", "well_level"]
    if add_watering_flag:
    feature_cols.append("watered")

    features = data[feature_cols].values.astype("float32")

    mean = features.mean(axis=0)
    std = features.std(axis=0)
    features = (features - mean) / std

    X, y = [], []
    for i in range(len(features) - window_size):
    X.append(features[i:i + window_size])
    y.append(features[i + window_size, 0])

    return np.asarray(X), np.asarray(y), mean, std


In [ ]:
window_size = 24
X, y, mean, std = load_data_from_csv(window_size)

# Build a sequential model using LSTM layers
model = Sequential([
    LSTM(64, input_shape=(window_size, X.shape[-1]), return_sequences=True),
    LSTM(32),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
# Train the model with 50 epochs and batch size of 32
model.save("../models/water_model.h5")